In [5]:
# Here we take care of paths.
# Make sure root project directory is named 'VESUVIUS_Challenge' for this to work

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in the root folder of the project
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [8]:
import torch
import monai
#from monai.visualize import matshow3d
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
#import tempfile
import shutil
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import Tuple, List
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Dataset import Vesuvius_Tile_Datamodule
from lit_models.Vesuvius_Lit_Model import Lit_Model
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn as nn

import torch.nn as nn
from functools import partial
import torchvision
import torch.nn.functional as F

from lit_models.scratch_models import FPNDecoder
from Models.PreBackbone_3D import PreBackbone_3D

In [16]:
smp.FPN(encoder_name='mit_b0', encoder_depth=5, encoder_weights='imagenet', decoder_pyramid_channels=256, decoder_segmentation_channels=128, decoder_merge_policy='cat', decoder_dropout=0.2, in_channels=3, classes=1, activation=None, upsampling=4, aux_params=None)




FPN(
  (encoder): MixVisionTransformerEncoder(
    (patch_embed1): OverlapPatchEmbed(
      (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed2): OverlapPatchEmbed(
      (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed3): OverlapPatchEmbed(
      (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
    )
    (patch_embed4): OverlapPatchEmbed(
      (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (block1): ModuleList(
      (0): Block(
        (norm1): LayerNorm((32,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (q): Linear(in_features=32, ou

In [3]:
z_dim = 32
out_channels = [4,8,16,32]

conv =  nn.Conv3d(in_channels=1,
                  out_channels=4,
                  kernel_size = (3, 1, 1),
                  stride=(2, 1, 1),
                  padding= (1, 0, 0)
                  )  # (B,C,H,W) -> (B, 1 C, H, W)

pool = nn.AvgPool3d(kernel_size = (2 ,1 ,1), stride=(2 ,1 ,1))

global_pool =  nn.AdaptiveAvgPool3d((1, None, None))

conv2 =  nn.Conv3d(in_channels=2,
                  out_channels=8,
                  kernel_size = (3, 1, 1),
                  stride=(2, 1, 1),
                  padding= (1, 0, 0)
                  )

conv3 =  nn.Conv3d(in_channels=4,
                  out_channels=16,
                  kernel_size = (3, 1, 1),
                  stride=(2, 1, 1),
                  padding= (1, 0, 0)
                  )





In [4]:

dummy  = torch.randn(5,1,z_dim,256,256)
print(dummy.shape)

torch.Size([5, 1, 32, 256, 256])


In [5]:
# LAYER 1
out = conv(dummy)
print(out.shape)
out = out.permute(0,2,1,3,4)
out = pool(out)
out = out.permute(0,2,1,3,4)
print(out.shape)

torch.Size([5, 4, 16, 256, 256])
torch.Size([5, 2, 16, 256, 256])


In [6]:
out = conv(dummy)

out = out.permute(0,2,1,3,4)
out = pool(out)
out = out.permute(0,2,1,3,4)

out = conv2(out)
out = out.permute(0,2,1,3,4)
out = pool(out)
out = out.permute(0,2,1,3,4)
out = conv3(out)


# FINAL
out = out.permute(0,2,1,3,4)
print(out.shape)
out = global_pool(out)
out = out.permute(0,2,1,3,4)
print(out.shape)

torch.Size([5, 4, 16, 256, 256])
torch.Size([5, 1, 4, 256, 256])


In [7]:
print(out.shape)

torch.Size([5, 1, 4, 256, 256])


In [8]:
batch_norm = torch.nn.BatchNorm3d( num_features=1, momentum=0.9)

In [9]:
batch_norm(out).shape

torch.Size([5, 1, 4, 256, 256])

In [10]:
m_dummy = torch.randn(5,48,256,256)
model_3d = PreBackbone_3D()
outs = model_3d(m_dummy)
print(outs.shape)

torch.Size([5, 3, 256, 256])


In [19]:
model = smp.Unet(encoder_name='mit_b2',
                 encoder_depth=5,
                 encoder_weights='imagenet', 
                 decoder_use_batchnorm=True, 
                 decoder_channels=(512, 256, 128, 64, 32,),
                 decoder_attention_type=None,
                 in_channels=3,
                 classes=1, activation=None, aux_params=None)





In [20]:
final_outs = model(outs)

In [21]:
final_outs.shape

torch.Size([5, 1, 256, 256])